In [1]:
!pip install datatable

In [1]:
import datatable as dt

In [2]:
data = dt.fread("../data/transactions_train.csv")

print("Dataset size:", data.shape)


Dataset size: (31788324, 5)


In [3]:
print(data[0, :])
# print(data[31788323, :])

   | t_dat       customer_id                                                       article_id      price  sales_channel_id
   | date32      str32                                                                  int32    float64             int32
-- + ----------  ----------------------------------------------------------------  ----------  ---------  ----------------
 0 | 2018-09-20  000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318   663713001  0.0508305                 2
[1 row x 5 columns]



In [4]:
from datetime import datetime, timedelta

last_date = data[-1,'t_dat']
print("Last date in dataset =", last_date)

# Calculate the date 7 days before
date_before_7_days = last_date - timedelta(days=7)

# Convert back to string if needed
date_before_7_days_str = date_before_7_days.strftime("%Y-%m-%d")

print("One week before =", date_before_7_days_str)



Last date in dataset = 2020-09-22
One week before = 2020-09-15


In [5]:
from datatable import f

test_data_size = -1

for i in range(data.nrows):
    date = data[-1 * (i+1) , 0]
    # print(date)
    if not (date > date_before_7_days):
        test_data_size = i
        break

print(test_data_size)


240311


In [6]:
train_data = data[:-1*test_data_size,:]
test_data = data[-1*test_data_size:,:]

print(train_data.head())


   | t_dat       customer_id                                                       article_id      price  sales_channel_id
   | date32      str32                                                                  int32    float64             int32
-- + ----------  ----------------------------------------------------------------  ----------  ---------  ----------------
 0 | 2018-09-20  000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318   663713001  0.0508305                 2
 1 | 2018-09-20  000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318   541518023  0.0304915                 2
 2 | 2018-09-20  00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2   505221004  0.0152373                 2
 3 | 2018-09-20  00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2   685687003  0.0169322                 2
 4 | 2018-09-20  00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2   685687004  0.0169322                 2
 5 | 2018-09-20 

In [7]:
frequency = train_data[:, dt.count(), dt.by(dt.f.article_id)]

# Printing the head of the frequency frame
print(frequency.head())

   | article_id  count
   |      int32  int64
-- + ----------  -----
 0 |  108775015  10841
 1 |  108775044   7247
 2 |  108775051    215
 3 |  110065001   1044
 4 |  110065002    539
 5 |  110065011   1090
 6 |  111565001   4882
 7 |  111565003     46
 8 |  111586001  14142
 9 |  111593001  13847
[10 rows x 2 columns]



In [8]:
frequency = frequency[:, :, dt.sort(-dt.f.count)]

# Select the top 'n' rows
n = 10  # Replace with the number of top elements you want
top_n_frequency = frequency[:n, :]

# Print the result
print(top_n_frequency)

   | article_id  count
   |      int32  int64
-- + ----------  -----
 0 |  706016001  49958
 1 |  706016002  34802
 2 |  372860001  31482
 3 |  610776002  30003
 4 |  759871002  26309
 5 |  464297007  24989
 6 |  372860002  24222
 7 |  610776001  22335
 8 |  399223001  22204
 9 |  720125001  21005
[10 rows x 2 columns]



In [9]:
most_pop_articles = top_n_frequency[:,0].to_list()[0]
print(most_pop_articles)

[706016001, 706016002, 372860001, 610776002, 759871002, 464297007, 372860002, 610776001, 399223001, 720125001]


In [10]:
df = train_data[:, :-2]

In [11]:
print(df.head())

   | t_dat       customer_id                                                       article_id
   | date32      str32                                                                  int32
-- + ----------  ----------------------------------------------------------------  ----------
 0 | 2018-09-20  000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318   663713001
 1 | 2018-09-20  000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318   541518023
 2 | 2018-09-20  00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2   505221004
 3 | 2018-09-20  00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2   685687003
 4 | 2018-09-20  00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2   685687004
 5 | 2018-09-20  00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2   685687001
 6 | 2018-09-20  00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2   505221001
 7 | 2018-09-20  00083cda041544b2fbb0e0d2905ad17da7cf1007526

In [44]:
# Assuming 'df' is your datatable Frame with 't_dat' as datetime.date objects and 'customer_id'

# Convert 't_dat' to a combined year-week identifier
df['year_week'] = dt.Frame([f"{date.year}-{date.isocalendar()[1]:02d}" for date in df['t_dat'].to_list()[0]])

# Group by customer_id and year_week, then count the purchases
weekly_purchases = df[:, dt.count(), dt.by('customer_id', 'year_week')]

# Calculate the average number of purchases per week for each customer
average_purchases = weekly_purchases[:, dt.mean(dt.f.count), dt.by('customer_id')]

# Print the head of the result
print(average_purchases.head())


: 

In [ ]:
average_purchases.to_csv("../data/avg_customer_purchases.csv")

In [12]:
avg_purchase_data = dt.fread("../data/avg_customer_purchases.csv")

print("Average weekly purchases frame size:", avg_purchase_data.shape)


Average weekly purchases frame size: (1329917, 2)


In [13]:
print(avg_purchase_data[0,:])

   | customer_id                                                         count
   | str32                                                             float64
-- + ----------------------------------------------------------------  -------
 0 | 00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657  1.88889
[1 row x 2 columns]



In [29]:


def pop_recommender(customer_id):
    

    # Select rows where the first column has the value 'x'
    num_purchases = round(avg_purchase_data[f[0] == customer_id, 1][0,0])


    return set(most_pop_articles[:num_purchases])


In [30]:
print(pop_recommender("00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657"))

{706016001, 706016002}


In [31]:
print(test_data.head())

   | t_dat       customer_id                                                       article_id       price  sales_channel_id
   | date32      str32                                                                  int32     float64             int32
-- + ----------  ----------------------------------------------------------------  ----------  ----------  ----------------
 0 | 2020-09-16  000fb6e772c5d0023892065e659963da90b1866035558ec16fca51b0dcfb7e59   786022008  0.0484407                  2
 1 | 2020-09-16  000fb6e772c5d0023892065e659963da90b1866035558ec16fca51b0dcfb7e59   913272003  0.0322881                  2
 2 | 2020-09-16  000fb6e772c5d0023892065e659963da90b1866035558ec16fca51b0dcfb7e59   889669006  0.0565085                  2
 3 | 2020-09-16  0010e8eb18f131e724d6997909af0808adbba057529edb1523944f7d4e02b4ce   237347060  0.0338814                  1
 4 | 2020-09-16  0010e8eb18f131e724d6997909af0808adbba057529edb1523944f7d4e02b4ce   562245001  0.0135424                  1
 5 | 202

In [24]:
from datatable import f

# customer_id_column = test_data[:, 'customer_id']

# Get unique values of 'customer_id'
unique_customer_ids = dt.unique(test_data['customer_id'])

# Convert to a Python list
# unique_customer_ids_list = unique_customer_ids.to_list()[0]

print(unique_customer_ids.head())

   | customer_id                                                     
   | str32                                                           
-- + ----------------------------------------------------------------
 0 | 00039306476aaf41a07fed942884f16b30abfa83a2a8bea972019098d6406793
 1 | 0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf4672f30b3e622fec55
 2 | 000493dd9fc463df1acc2081450c9e75ef8e87d5dd17ed6396773839f6bf71a9
 3 | 000525e3fe01600d717da8423643a8303390a055c578ed8a97256600baf54565
 4 | 00077dbd5c4a4991e092e63893ccf29294a9d5c46e85010e95f2fc10bf9437a4
 5 | 0008daf19b2a7cd6fa00836c717aa0f143c83d88c35e0269a28cbff53664205a
 6 | 000fb6e772c5d0023892065e659963da90b1866035558ec16fca51b0dcfb7e59
 7 | 0010e8eb18f131e724d6997909af0808adbba057529edb1523944f7d4e02b4ce
 8 | 001143bec624de9df82bd687babb382e00b0285e2371cc9120de5d42c48f1076
 9 | 001324f693acaea0dea35333ba00ccddd0162d8bc81e76865034fca36cbb89c8
[10 rows x 1 column]



In [28]:
test_purchases_made = {}
unique_customer_ids_list = unique_customer_ids.to_list()[0]

for customer_id in unique_customer_ids_list:
    customer_purchases = test_data[f.customer_id == customer_id, 'article_id'].to_list()[0]
    # print(customer_purchases)
    # break
    test_purchases_made[customer_id] = set(customer_purchases)
    # print(test_purchases_made)
    # break

In [33]:
correct_pred = 0

for customer_id in unique_customer_ids_list:
    try:
        pred_purchases = pop_recommender(customer_id)
    except:
        continue

    target_purchases = test_purchases_made[customer_id]
    common_values = pred_purchases.intersection(target_purchases)
    total_values = pred_purchases.union(target_purchases)
    # Count the number of common elements
    num_common_values = len(common_values)
    num_total_values = len(total_values)

    weight = (1.0 * num_common_values) / num_total_values
    correct_pred += weight

accuracy = correct_pred / len(unique_customer_ids_list)
print("POP Accuracy =", accuracy)




POP Accuracy = 0.0017288506679628633
